In [3]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
import warnings 
warnings.simplefilter("ignore")

In [9]:
df_1 = pd.read_excel(r"C:\Users\Win10\Desktop\zip\MP\WOS_forecast\08_02_24\Sales.xlsx")
df_2 = pd.read_excel(r"C:\Users\Win10\Desktop\zip\MP\WOS_forecast\08_02_24\SalesG_2022.xlsx")


In [11]:
df_1.columns

Index(['Filename', 'CustomerID', 'CustomerCode', 'CustomerName', 'InvoiceDate',
       'InvoiceNo', 'InvoiceOrCredit', 'InvoiceHeaderSalesRep', 'RepTeam',
       'OrderNo', 'InvoiceLineStockLocationID', 'StockLocationName',
       'VariantCode', 'VariantDescription', 'Category', 'CRQty', 'CRRSP',
       'CRLineDiscount', 'CRItemPrice', 'Price per', 'CRLineNet', 'CRGoodsNet',
       'CRDelNet', 'CRDissurNet', 'CRNet', 'm² per Product', 'm² Sold'],
      dtype='object')

In [14]:
df_1.isnull().sum()

Filename                          0
CustomerID                        0
CustomerCode                      0
CustomerName                      0
InvoiceDate                       0
InvoiceNo                         0
InvoiceOrCredit                   0
InvoiceHeaderSalesRep          9919
RepTeam                       16359
OrderNo                           0
InvoiceLineStockLocationID        0
StockLocationName                 0
VariantCode                       0
VariantDescription                0
Category                        681
CRQty                             0
CRRSP                             0
CRLineDiscount                    0
CRItemPrice                       0
Price per                         0
CRLineNet                         0
CRGoodsNet                        0
CRDelNet                          0
CRDissurNet                       0
CRNet                             0
m² per Product                 6850
m² Sold                        6850
dtype: int64

In [29]:
df_1 = df_1[~df_1['m² Sold'].isnull()]

In [30]:
df_1.isnull().sum()

Filename                          0
CustomerID                        0
CustomerCode                      0
CustomerName                      0
InvoiceDate                       0
InvoiceNo                         0
InvoiceOrCredit                   0
InvoiceHeaderSalesRep          7522
RepTeam                       12775
OrderNo                           0
InvoiceLineStockLocationID        0
StockLocationName                 0
VariantCode                       0
VariantDescription                0
Category                        270
CRQty                             0
CRRSP                             0
CRLineDiscount                    0
CRItemPrice                       0
Price per                         0
CRLineNet                         0
CRGoodsNet                        0
CRDelNet                          0
CRDissurNet                       0
CRNet                             0
m² per Product                    0
m² Sold                           0
dtype: int64

In [34]:
df_1[["VariantCode","Category"]]

,VariantCode,Category
0,JCPE6090S,Porcelain > India > Jewell Range
1,JCPE6090,Porcelain > India > Jewell Range
2,CGSC6515B,Sandstone > Accessories > Cladding > Castle Grey
3,CGSC6515,Sandstone > Accessories > Cladding > Castle Grey
4,SIPE6090S,Porcelain > India > Station Range
...,...,...
32665,MNWL0215S,Sandstone > Old Rectory Collection > Walling >...
32684,SBSP6060S,Sandstone > Gardenstone Collection > Paving > ...
32685,SBSP6060,Sandstone > Gardenstone Collection > Paving > ...
32686,SBSP6060,Sandstone > Gardenstone Collection > Paving > ...


In [39]:
category_map = (
    df_1[df_1['Category'].notnull()]
    .groupby('VariantCode')['Category']
    .agg(lambda x: x.mode()[0])  # Most frequent category
    .to_dict()
)
category_map

{'ABPE2020': 'Porcelain > Spain > Small Size Series',
 'ABPE2020-12P': 'Porcelain > Spain > Small Size Series',
 'ABPE2020S': 'Porcelain > Spain > Small Size Series',
 'ABPE2030-8P': 'Porcelain > Spain > Small Size Series',
 'ABPE2030S': 'Porcelain > Spain > Small Size Series',
 'ABSP1952': 'Sandstone > Gardenstone Collection',
 'ABSP3030S': 'Sandstone > Gardenstone Collection > Paving > Autumn Brown',
 'ABSP3060S': 'Sandstone > Gardenstone Collection > Paving > Autumn Brown',
 'ABSP6060': 'Sandstone > Gardenstone Collection > Paving > Autumn Brown',
 'ABSP6060S': 'Sandstone > Gardenstone Collection > Paving > Autumn Brown',
 'ABSP6090': 'Sandstone > Gardenstone Collection > Paving > Autumn Brown',
 'ABSP6090S': 'Sandstone > Gardenstone Collection > Paving > Autumn Brown',
 'ACPE3012': 'Porcelain > India',
 'ACPE3012S': 'Porcelain > India',
 'AGPE2020': 'Porcelain > Spain > Small Size Series',
 'AGPE2020S': 'Porcelain > Spain > Small Size Series',
 'AGPE2030': 'Porcelain > Spain > Smal

In [ ]:
df_1['category'] = df_1.apply(lambda row: category_map.get(row['VariantCode']) if pd.isnull(row['Category']) else row['Category'],axis=1)

In [49]:
df_1['Category'].fillna(df_1[df_1['Category'].isnull()]['VariantCode'],inplace=True)

In [53]:
df_1.drop(['RepTeam','InvoiceHeaderSalesRep'],axis=1,inplace=True)

In [63]:
df_1.columns
df_1.columns = list(map(lambda x: x.upper(),df_1.columns))

In [70]:
df_1[df_1['M² SOLD'] < 0].describe(percentiles=[0.1,0.4,0.8,1])

,CUSTOMERID,INVOICEDATE,INVOICENO,INVOICELINESTOCKLOCATIONID,CRQTY,CRRSP,CRLINEDISCOUNT,CRITEMPRICE,PRICE PER,CRLINENET,CRGOODSNET,CRDELNET,CRDISSURNET,CRNET,M² PER PRODUCT,M² SOLD
count,834.000000,834,834.000000,834.000000,834.000000,834.000000,834.000000,834.000000,834.0,834.000000,834.000000,834.000000,834.000000,834.000000,834.000000,834.000000
mean,2303.428058,2023-07-22 05:10:47.482014464,106270.098321,3.465228,-13.483213,-0.530588,285.275097,-285.805685,1.0,-737.919568,-717.199053,-2.307506,13.245372,-706.261187,14.598195,-35.836505
min,1579.000000,2023-01-07 00:00:00,98307.000000,3.000000,-1536.000000,-442.510000,-112.680000,-16543.000000,1.0,-16543.000000,-17778.900000,-100.000000,-263.680000,-17778.900000,0.010000,-683.200000
10%,1650.000000,2023-04-01 00:00:00,100790.000000,3.000000,-14.700000,0.000000,8.010800,-611.391000,1.0,-1922.871000,-1542.420000,0.000000,0.000000,-1488.060000,0.500000,-77.760000
40%,1996.200000,2023-06-30 00:00:00,105331.400000,3.000000,-1.000000,0.000000,90.360000,-267.306000,1.0,-354.632000,-290.910000,0.000000,0.000000,-272.900000,15.200000,-19.680000
50%,2047.000000,2023-07-26 00:00:00,106752.500000,3.000000,-1.000000,0.000000,171.440000,-173.485000,1.0,-247.500000,-144.290000,0.000000,0.000000,-149.350000,15.300000,-19.520000
80%,2971.000000,2023-10-09 00:00:00,110078.200000,3.000000,-1.000000,0.000000,460.114600,-20.162000,1.0,-50.946000,0.000000,0.000000,0.000000,0.000000,24.480000,-10.048000
100%,3683.000000,2023-12-19 00:00:00,112131.000000,10.000000,-1.000000,0.000000,16543.000000,0.000000,1.0,0.000000,0.000000,0.000000,896.530000,0.000000,46.200000,-0.010000
max,3683.000000,2023-12-19 00:00:00,112131.000000,10.000000,-1.000000,0.000000,16543.000000,0.000000,1.0,0.000000,0.000000,0.000000,896.530000,0.000000,46.200000,-0.010000
std,592.346671,NaN,3770.602855,1.602628,72.918760,15.322868,700.445949,700.311724,0.0,1496.431248,1917.371199,12.575887,67.959755,1911.355343,9.230748,70.441534


In [73]:
df_1.CATEGORY.value_counts()

CATEGORY
Porcelain > India > Station Range                                           3845
Sandstone > Gardenstone Collection > Paving > Pure Grey                     2521
Sandstone > Gardenstone Collection > Paving > Raj Blend                     1674
Special Orders > MKM > Porcelain                                            1411
Porcelain > India > Exquisite Range                                         1183
                                                                            ... 
Discontinued > Porcelain > Italy > Mixed Size Series                           1
Discontinued > Sandstone > Old Rectory Collection > Pavers > York Green        1
Discontinued > Sandstone > Old Rectory Collection > Cobbles > Modak Rose       1
Digby Stone > Porcelain > Country                                              1
Special Orders > EHSmith > Porcelain                                           1
Name: count, Length: 234, dtype: int64

In [74]:
df_1.info()

<class 'pandas.core.frame.DataFrame'>
Index: 25838 entries, 0 to 32687
Data columns (total 25 columns):
 #   Column                      Non-Null Count  Dtype         
---  ------                      --------------  -----         
 0   FILENAME                    25838 non-null  object        
 1   CUSTOMERID                  25838 non-null  int64         
 2   CUSTOMERCODE                25838 non-null  object        
 3   CUSTOMERNAME                25838 non-null  object        
 4   INVOICEDATE                 25838 non-null  datetime64[ns]
 5   INVOICENO                   25838 non-null  int64         
 6   INVOICEORCREDIT             25838 non-null  object        
 7   ORDERNO                     25838 non-null  object        
 8   INVOICELINESTOCKLOCATIONID  25838 non-null  int64         
 9   STOCKLOCATIONNAME           25838 non-null  object        
 10  VARIANTCODE                 25838 non-null  object        
 11  VARIANTDESCRIPTION          25838 non-null  object        


In [76]:
df_1.INVOICEDATE = pd.to_datetime(df_1.INVOICEDATE)

In [79]:
df_1['MONTH'] = df_1.INVOICEDATE.dt.to_period("M")

In [81]:
df_1.MONTH = df_1.MONTH.dt.to_timestamp()
df_1.MONTH.value_counts()

MONTH
2023-05-01    3317
2023-06-01    2968
2023-07-01    2946
2023-03-01    2587
2023-04-01    2575
2023-08-01    2278
2023-09-01    2164
2023-01-01    1815
2023-10-01    1796
2023-02-01    1680
2023-11-01    1159
2023-12-01     553
Name: count, dtype: int64

In [90]:
df_1

,FILENAME,CUSTOMERID,CUSTOMERCODE,CUSTOMERNAME,INVOICEDATE,INVOICENO,INVOICEORCREDIT,ORDERNO,INVOICELINESTOCKLOCATIONID,STOCKLOCATIONNAME,...,CRITEMPRICE,PRICE PER,CRLINENET,CRGOODSNET,CRDELNET,CRDISSURNET,CRNET,M² PER PRODUCT,M² SOLD,MONTH
0,05-0008-012 - Invoiced Lines By Category Report,2104,BSB041,Huws Gray Ltd - Grantham - BSB041,2023-01-04,98153,Invoice,SO99807,3,Global Stone,...,18.860,1,377.20,1282.48,0.0,0.00,1282.48,1.0000,20.0000,2023-01-01
1,05-0008-012 - Invoiced Lines By Category Report,2104,BSB041,Huws Gray Ltd - Grantham - BSB041,2023-01-04,98153,Invoice,SO99807,3,Global Stone,...,905.280,1,905.28,0.00,0.0,0.00,0.00,25.9200,25.9200,2023-01-01
2,05-0008-012 - Invoiced Lines By Category Report,2095,BSB032,Huws Gray Ltd - Earl Shilton - BSB032,2023-01-04,98154,Invoice,SO99850,3,Global Stone,...,55.270,1,110.54,1187.20,60.0,0.00,1247.20,0.5400,1.0800,2023-01-01
3,05-0008-012 - Invoiced Lines By Category Report,2095,BSB032,Huws Gray Ltd - Earl Shilton - BSB032,2023-01-04,98154,Invoice,SO99850,3,Global Stone,...,1076.660,1,1076.66,0.00,0.0,0.00,0.00,12.9600,12.9600,2023-01-01
4,05-0008-012 - Invoiced Lines By Category Report,2094,BSB031,Huws Gray Ltd - Dover - BSB031,2023-01-04,98155,Invoice,SO99886,3,Global Stone,...,22.990,1,275.88,1180.93,0.0,0.00,1180.93,0.5400,6.4800,2023-01-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32665,05-0008-012 - Invoiced Lines By Category Report,1632,COS001,Country Supplies-High Wycombe-COS001,2023-12-19,112116,Invoice,SO119261,3,Global Stone,...,0.000,1,0.00,0.00,0.0,0.00,0.00,0.0215,0.0215,2023-12-01
32684,05-0008-012 - Invoiced Lines By Category Report,2063,MKM072,MKM Building Supplies Ltd - York - MKM072,2023-12-19,112130,Credit,SO119682,3,Global Stone,...,-8.040,1,-321.60,-1125.46,0.0,158.81,-966.65,0.3600,-14.4000,2023-12-01
32685,05-0008-012 - Invoiced Lines By Category Report,2063,MKM072,MKM Building Supplies Ltd - York - MKM072,2023-12-19,112130,Credit,SO119682,3,Global Stone,...,-401.925,1,-401.93,0.00,0.0,0.00,0.00,18.0000,-18.0000,2023-12-01
32686,05-0008-012 - Invoiced Lines By Category Report,2063,MKM072,MKM Building Supplies Ltd - York - MKM072,2023-12-19,112130,Credit,SO119682,3,Global Stone,...,-401.925,1,-401.93,0.00,0.0,0.00,0.00,18.0000,-18.0000,2023-12-01


In [96]:
df_1.describe(include=["O"])

,FILENAME,CUSTOMERCODE,CUSTOMERNAME,INVOICEORCREDIT,ORDERNO,STOCKLOCATIONNAME,VARIANTCODE,VARIANTDESCRIPTION,CATEGORY
count,25838,25838,25838,25838,25838,25838,25838,25838,25838
unique,1,1007,1005,2,11391,5,1175,1175,234
top,05-0008-012 - Invoiced Lines By Category Report,IBM001,IBMG Ltd-Polegate-IBM001,Invoice,SO107806,Global Stone,PGSP1952,Pure Grey Sandstone 65pcs 19.52m2 PP,Porcelain > India > Station Range
freq,25838,877,877,24982,57,19040,1022,1022,3845


In [ ]:
df_group = df_1.groupby(["MONTH","VARIANTCODE"]).agg({'VARIANTCODE':'count','M² SOLD':'sum'})

In [114]:
df_group.columns = ['VARIANTCODE_count', 'M² SOLD']

In [115]:
df_group.reset_index()

,MONTH,VARIANTCODE,VARIANTCODE_count,M² SOLD
0,2023-01-01,ABPE2020,1,0.2400
1,2023-01-01,ABPE2020S,2,0.0800
2,2023-01-01,ABPE2030S,1,0.0600
3,2023-01-01,ABSP1952,2,156.1600
4,2023-01-01,ABSP3030S,2,0.8100
...,...,...,...,...
5830,2023-12-01,YGSP6090,1,60.4800
5831,2023-12-01,YGWL0215,3,81.0000
5832,2023-12-01,YGWL0215S,1,0.0215
5833,2023-12-01,YGWL0325,2,13.5000


In [116]:
df_group.describe()

,VARIANTCODE_count,M² SOLD
count,5835.000000,5835.000000
mean,4.428106,221.896312
std,8.263784,1095.045426
min,1.000000,-491.520000
25%,1.000000,1.080000
50%,2.000000,16.200000
75%,4.000000,96.900000
max,126.000000,29377.600000


In [119]:
df_group[df_group.VARIANTCODE_count >0].sort_values(by="M² SOLD",ascending=False)

,,VARIANTCODE_count,M² SOLD
MONTH,VARIANTCODE,,
2023-08-01,PGSP1952,108,29377.60
2023-07-01,PGSP1952,126,26195.84
2023-06-01,PGSP1952,126,22545.60
2023-01-01,PGSP1952,59,21647.68
2023-05-01,PGSP1952,118,20456.96
...,...,...,...
2023-09-01,BQPE4080S,1,-199.68
2023-11-01,DPNRSRTSS60X60PART,1,-230.40
2023-07-01,D7GRSP1952,5,-292.80


In [122]:
df_group.T

MONTH             2023-01-01                                                   \
VARIANTCODE         ABPE2020 ABPE2020S ABPE2030S ABSP1952 ABSP3030S ABSP3060S   
VARIANTCODE_count       1.00      2.00      1.00     2.00      2.00      2.00   
M² SOLD                 0.24      0.08      0.06   156.16      0.81      1.62   

MONTH                                                    ... 2023-12-01  \
VARIANTCODE       ABSP6060 ABSP6060S ABSP6090 ABSP6090S  ...   TUCP2110   
VARIANTCODE_count      1.0       3.0     1.00      2.00  ...        3.0   
M² SOLD               54.0       3.6    45.36      4.86  ...       45.6   

MONTH                                                                     \
VARIANTCODE       USPE4080S YGCB1071 YGSC2800 YGSP1689 YGSP6090 YGWL0215   
VARIANTCODE_count       1.0     3.00      1.0     3.00     1.00      3.0   
M² SOLD               307.2    42.84      0.0   371.58    60.48     81.0   

MONTH                                           
VARIANTCODE       YGWL0215S YGWL0325 YGWL0325S  
VARIANTCODE_count    1.0000      2.0    1.0000  
M² SOLD              0.0215     13.5    0.0325  

[2 rows x 5835 columns]